In [1]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient

import random, time, datetime 
import pandas as pd
import json

In [2]:
SHADOW_CLIENT="A0186745N_1"
HOST_NAME="a2zdmpkl8brfji-ats.iot.us-west-2.amazonaws.com"
ROOT_CA="Amazon_Root_CA_1.pem"
PRIVATE_KEY="703b1aaaac-private.pem.key"
CERT_FILE="703b1aaaac-certificate.pem.crt"
SHADOW_HANDLER="A0186745N_1"

In [3]:
def myShadowUpdateCallback(payload,responseStatus,token):
    print()
    print('UPDATE:$aws/things/'+SHADOW_HANDLER+
          '/shadow/update/#')
    #print("payload="+payload)
    #print("responseStatus="+responseStatus)
    #print("token="+token)

In [4]:
def customShadowCallback_delete(payload, responseStatus, token):
    if responseStatus == "timeout":
        print("Delete request " + token + " time out!")
    if responseStatus == "accepted":
        print("Delete request with token " + token + " accepted!")
    if responseStatus == "rejected":
        print("Delete request with token " + token + " rejected!")

In [5]:
def customShadowCallback_Delta(payload, responseStatus, token):
    # payload is a JSON string which will be parsed by jason lib
    print(responseStatus)
    print(payload)
    payloadDict = json.loads(payload)
    print("++++++++ Get DELTA data +++++++++++")
    desiredStatus = str(payloadDict["reported"])
    print("desired status: " + desiredStatus)
    print("version: " + str(payloadDict["version"]))

In [6]:
def customShadowCallback_Get(payload, responseStatus, token):
    print("responseStatus: " + responseStatus)
    print("payload: " + payload)
    payloadDict = json.loads(payload)
    # {"state":{"desired":{"light":37},"delta":{"light":37}},"metadata":{"desired":{"light":{"timestamp":1533888405}}},"version":54

In [7]:
print("connect to AWS IoT")
myShadowClient=AWSIoTMQTTShadowClient(SHADOW_CLIENT)
myShadowClient.configureEndpoint(HOST_NAME,8883)
myShadowClient.configureCredentials(ROOT_CA,PRIVATE_KEY,CERT_FILE)
# myMQTTClient=AWSIoTMQTTClient(SHADOW_CLIENT)
# myMQTTClient.configureEndpoint(HOST_NAME, 8883)
# myMQTTClient = myShadowClient.getMQTTConnection()
# myMQTTClient.configureOfflinePublishQueueing(-1)
myShadowClient.configureConnectDisconnectTimeout(10)
myShadowClient.configureMQTTOperationTimeout(10)
myShadowClient.connect()
myDeviceShadow=myShadowClient.createShadowHandlerWithName(SHADOW_HANDLER,True)

connect to AWS IoT


In [8]:
#%% load library
import gc
#import resource
import rope
#import operator
import numpy as np
import pandas as pd

In [9]:
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set(style="white", color_codes=True)

#import xgboost as xgb
#from xgboost.sklearn import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV

import warnings
warnings.filterwarnings("ignore")

In [10]:
#%% Load data
print('Start loading data ...')

# Load and combine training data
train_1 = pd.read_csv('train_FD001.txt', delim_whitespace=True, header=None)
#train_2 = pd.read_csv('./input/train_FD002.txt', delim_whitespace=True, header=None)
#train_3 = pd.read_csv('./input/train_FD003.txt', delim_whitespace=True, header=None)
#train_4 = pd.read_csv('./input/train_FD004.txt', delim_whitespace=True, header=None)

train=train_1

Start loading data ...


In [11]:
# Check dimension
print('Dimension of train_FD001: ', train.shape)

Dimension of train_FD001:  (20631, 26)


In [12]:
sensor_name = ['s'+ str(i) for i in range(1,22)]
train.columns = ['id', 'te', 'os1', 'os2', 'os3'] + sensor_name
train['id'] = train_1['id'].map(lambda s: 'FD001_'+str(s))
train['Maltricnum']='A0186745N'

In [17]:
import time
for i in range(5):
    test=train.loc[[i]] 
    now = datetime.datetime.utcnow()
    now='UTC'+str(now)
    test['timestamp']=now
    test.index=["reported"]
    test=test.to_json(orient='index')
    test='{"state":'+test+'}'
    jsonPayload=test    
    
    myDeviceShadow.shadowUpdate(jsonPayload,myShadowUpdateCallback, 5)
    time.sleep(10)


UPDATE:$aws/things/A0186745N_1/shadow/update/#

UPDATE:$aws/things/A0186745N_1/shadow/update/#

UPDATE:$aws/things/A0186745N_1/shadow/update/#

UPDATE:$aws/things/A0186745N_1/shadow/update/#

UPDATE:$aws/things/A0186745N_1/shadow/update/#


In [21]:
print(test)

{"state":{"reported":{"id":"FD002_1","te":63,"os1":41.9992,"os2":0.8413,"os3":100.0,"s1":445.0,"s2":550.1,"s3":1355.17,"s4":1123.54,"s5":3.91,"s6":5.71,"s7":138.41,"s8":2211.53,"s9":8311.01,"s10":1.02,"s11":42.27,"s12":130.96,"s13":2387.6,"s14":8066.56,"s15":9.362,"s16":0.02,"s17":331,"s18":2212,"s19":100.0,"s20":10.36,"s21":6.3852,"Maltricnum":"A0184800J","timestamp":"UTC2019-09-17 05:41:22.036662"}}}


In [15]:
myDeviceShadow.shadowDelete( customShadowCallback_delete, 5)

'aa237f28-e6bb-405f-bc83-2cf507ea9c58'

Delete request with token aa237f28-e6bb-405f-bc83-2cf507ea9c58 accepted!
